<a href="https://colab.research.google.com/github/vanessabrolim/comfort_analysis/blob/master/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from geopy.distance import vincenty
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importação de dados

In [0]:
df = pd.read_pickle('drive/My Drive/Udesc/df.pkl')

In [0]:
list(df)

['DATE_TIME',
 'LATITUDE',
 'LONGITUDE',
 'ENGINE_RPM',
 'ENGINE_LOAD',
 'INTAKE_MANIFOLD_PRESSURE',
 'ACCELERATOR_PEDAL_D',
 'SPEED',
 'GYROSCOPE_X_RAW',
 'GYROSCOPE_Y_RAW',
 'GYROSCOPE_Z_RAW',
 'ACCELEROMETER_X_LONG_AVG',
 'ACCELEROMETER_Y_LONG_AVG',
 'ACCELEROMETER_Z_LONG_AVG',
 'ACCELEROMETER_X_LONG_STD',
 'ACCELEROMETER_Y_LONG_STD',
 'ACCELEROMETER_Z_LONG_STD']

In [0]:
df.head()

,DATE_TIME,LATITUDE,LONGITUDE,ENGINE_RPM,ENGINE_LOAD,INTAKE_MANIFOLD_PRESSURE,ACCELERATOR_PEDAL_D,SPEED,GYROSCOPE_X_RAW,GYROSCOPE_Y_RAW,GYROSCOPE_Z_RAW,ACCELEROMETER_X_LONG_AVG,ACCELEROMETER_Y_LONG_AVG,ACCELEROMETER_Z_LONG_AVG,ACCELEROMETER_X_LONG_STD,ACCELEROMETER_Y_LONG_STD,ACCELEROMETER_Z_LONG_STD
1,2017-09-15 00:20:28.302,-26.272034,-48.850364,NaN,NaN,NaN,NaN,0.127404,-1.29,-0.02,-2.58,1.018600,0.081000,0.115000,0.006020,0.002608,0.001897
2,2017-09-15 00:20:28.802,-26.272033,-48.850365,NaN,NaN,NaN,NaN,0.061108,-1.26,-0.13,-2.30,1.018000,0.080900,0.115500,0.005040,0.002587,0.004177
3,2017-09-15 00:20:29.302,-26.272033,-48.850366,NaN,NaN,NaN,NaN,0.009961,-1.35,-0.24,-2.41,1.017867,0.080333,0.115933,0.004272,0.002821,0.003750
4,2017-09-15 00:20:29.802,-26.272033,-48.850367,NaN,NaN,NaN,NaN,0.014452,-1.22,-0.29,-2.58,1.017500,0.079750,0.115750,0.003814,0.002947,0.003832
5,2017-09-15 00:20:30.302,-26.272033,-48.850368,NaN,NaN,NaN,NaN,0.060168,-1.10,-0.29,-2.24,1.017760,0.079920,0.116040,0.003798,0.002911,0.004025


### Retirada de Stay Point

In [0]:
df.sort_values(by=['DATE_TIME'])

,DATE_TIME,LATITUDE,LONGITUDE,ENGINE_RPM,ENGINE_LOAD,INTAKE_MANIFOLD_PRESSURE,ACCELERATOR_PEDAL_D,SPEED,GYROSCOPE_X_RAW,GYROSCOPE_Y_RAW,GYROSCOPE_Z_RAW,ACCELEROMETER_X_LONG_AVG,ACCELEROMETER_Y_LONG_AVG,ACCELEROMETER_Z_LONG_AVG,ACCELEROMETER_X_LONG_STD,ACCELEROMETER_Y_LONG_STD,ACCELEROMETER_Z_LONG_STD
1,2017-09-15 00:20:28.302,-26.272034,-48.850364,NaN,NaN,NaN,NaN,0.127404,-1.29,-0.02,-2.58,1.018600,0.081000,0.115000,0.006020,0.002608,0.001897
2,2017-09-15 00:20:28.802,-26.272033,-48.850365,NaN,NaN,NaN,NaN,0.061108,-1.26,-0.13,-2.30,1.018000,0.080900,0.115500,0.005040,0.002587,0.004177
3,2017-09-15 00:20:29.302,-26.272033,-48.850366,NaN,NaN,NaN,NaN,0.009961,-1.35,-0.24,-2.41,1.017867,0.080333,0.115933,0.004272,0.002821,0.003750
4,2017-09-15 00:20:29.802,-26.272033,-48.850367,NaN,NaN,NaN,NaN,0.014452,-1.22,-0.29,-2.58,1.017500,0.079750,0.115750,0.003814,0.002947,0.003832
5,2017-09-15 00:20:30.302,-26.272033,-48.850368,NaN,NaN,NaN,NaN,0.060168,-1.10,-0.29,-2.24,1.017760,0.079920,0.116040,0.003798,0.002911,0.004025
6,2017-09-15 00:20:30.802,-26.272033,-48.850369,NaN,NaN,NaN,NaN,0.260967,-1.23,-0.13,-2.42,1.017767,0.079167,0.116200,0.003853,0.003416,0.004679
7,2017-09-15 00:20:31.302,-26.272033,-48.850370,NaN,NaN,NaN,NaN,0.386028,-1.25,-0.12,-2.45,1.017514,0.079143,0.116200,0.003683,0.004511,0.004541
8,2017-09-15 00:20:31.802,-26.272033,-48.850371,NaN,NaN,NaN,NaN,0.222701,-1.17,-0.21,-2.22,1.017750,0.078950,0.115975,0.003576,0.004483,0.004777
9,2017-09-15 00:20:32.302,-26.272033,-48.850371,NaN,NaN,NaN,NaN,0.092625,-1.38,-0.37,-2.35,1.018244,0.078800,0.115822,0.003962,0.005655,0.004995
10,2017-09-15 00:20:32.802,-26.272033,-48.850372,NaN,NaN,NaN,NaN,0.086138,-1.23,-0.22,-2.51,1.018440,0.078440,0.115840,0.004100,0.006287,0.004965


In [0]:
last_point = (df['LATITUDE'].iloc[0], df['LONGITUDE'].iloc[0])
last_moment = df['DATE_TIME'].iloc[0]


In [0]:
last_point

(-26.2720336, -48.8503641)

In [0]:
last_moment

Timestamp('2017-09-15 00:20:28.302000')

In [0]:
min_distance = 500
min_interval = 0.1
max_interval = 4
valid = []
stay_point = []

In [0]:
for index, row in df.iterrows():
  current_moment = row['DATE_TIME']
  current_point = (row['LATITUDE'], row['LONGITUDE'])
  distance = vincenty(current_point, last_point).meters
  interval = (current_moment - last_moment).total_seconds()
  if (interval < max_interval and interval > min_interval):
    valid.append(row)
    last_moment = current_moment
    last_point = current_point
  else: 
    stay_point.append(row)

In [0]:
len(valid)

625500

In [0]:
len(stay_point)

826211

In [0]:
df.count()

DATE_TIME                   1451711
LATITUDE                    1451711
LONGITUDE                   1451711
ENGINE_RPM                   363508
ENGINE_LOAD                  363711
INTAKE_MANIFOLD_PRESSURE     363420
ACCELERATOR_PEDAL_D          363819
SPEED                       1451711
GYROSCOPE_X_RAW             1451708
GYROSCOPE_Y_RAW             1451708
GYROSCOPE_Z_RAW             1451708
ACCELEROMETER_X_LONG_AVG    1451711
ACCELEROMETER_Y_LONG_AVG    1451711
ACCELEROMETER_Z_LONG_AVG    1451711
ACCELEROMETER_X_LONG_STD    1451711
ACCELEROMETER_Y_LONG_STD    1451711
ACCELEROMETER_Z_LONG_STD    1451711
dtype: int64

In [0]:
df_valid = pd.DataFrame(valid)
df_stay = pd.DataFrame(stay_point)

In [0]:
df_valid.to_pickle("./df_valid.pkl")